# Step 7
You should now have deployed the Analysis Lambda service. We can use the code in this notebook to test the service. 

## 1. View Bucket Contents
Run the code below to view the contents of your bucket.

In [ ]:
// load the AWS SDK
AWS = require('aws-sdk')
support = require('/home/dev/support')

// create an S3 object to call
s3 = new AWS.S3()

// configure parameters
params = {
    Bucket: process.env.MY_BUCKET_NAME
}

s3.listObjects(params, function(err, data) {
  support.printS3Objects(data.Contents)
})
$$.clear()

## 2. Post a Message
We will again post a message to our Crawler queue, only this time because the Analysis Service is deployed, it will pick up on a second message that is posted from the Crawler Service to the Analysis queue.

Go ahead and run the code below.

In [ ]:
// load the AWS SDK
AWS = require('aws-sdk')

// create an SQS object to call
sqs = new AWS.SQS()

// construct Queue URL
queueUrl = `https://sqs.${process.env.AWS_REGION}.amazonaws.com/${process.env.AWS_ACCOUNT_ID}/${process.env.MY_CRAWLER_QUEUE}`

// construct message parameters
params = {
  MessageBody: JSON.stringify({action: 'download', msg: {url: 'http://ai-as-a-service.s3-website-eu-west-1.amazonaws.com'}}),
  QueueUrl: queueUrl
}

// send the message
sqs.sendMessage(params, (err, data) => {
    if (err) {
        console.log(`QUEUE ERROR: ${err}`)
    }
    else {
      console.log(`message id: ${data.MessageId}`)        
    }
})
$$.clear()

# 3. Check the Results
If you rerun the code in 1. at the top of this notebook, you should see some additoinal files have been downloaded if our crawler executed successfully. You can confirm this if you look in the AWS S3 console.

If you look at the output you should see that there are a number of downloaded images and also a file `status.json`. The status file is generated by the crawler. Let's check the contents of this file by running the code below:

You will need to replace the `Key` entry in the parameters object with your own key path. You can copy and paste this from the above output.

In [ ]:
// load the AWS SDK
AWS = require('aws-sdk')

// create an S3 object to call
s3 = new AWS.S3()

// configure parameters
params = {
    Bucket: process.env.MY_BUCKET_NAME,
    Key: 'lob2ic9ac.ai-as-a-service.s3-website-eu-west-1.amazonaws.com/status.json' // <-- replace with your key path to status.json
}

s3.getObject(params, function(err, data) {
  if (err) {
    console.log(`QUEUE ERROR: ${err}`)
  }
  else {
    console.log(JSON.stringify(JSON.parse(data.Body.toString()), null, 2))
  }
})
$$.clear()

In addition to the `downloadResults` block you should also see an `analysisResults` block. This holds the output of the analysis service inclduing the outputs from Rekognition and also the data for a word cloud of the results.

Now that you have successfully deployed the Asynchronous services, it's time to finish off our system by deploying a UI.